# Training Demo for DeepReaction

This notebook demonstrates how to train a molecular reaction prediction model using the DeepReaction framework with a simplified unified configuration interface.

## 1. Import Required Libraries

In [15]:
import os
import sys
import torch
import numpy as np
from pathlib import Path

# Import from deepreaction package using the new unified interface
from deepreaction import ReactionTrainer, ReactionDataset, Config

## 2. Define Training Parameters

All parameters are defined in a single dictionary for simplicity.

In [16]:
# Define all parameters in a single dictionary
params = {
    # Dataset parameters
    'dataset': 'XTB',
    'readout': 'mean',
    'dataset_root': './dataset/DATASET_DA_F',  # Adjust path if needed
    'dataset_csv': './dataset/DATASET_DA_F/dataset_xtb_final.csv', # Adjust path if needed
    'train_ratio': 0.8,
    'val_ratio': 0.1,
    'test_ratio': 0.1,
    'target_fields': ['G(TS)', 'DrG'],
    'target_weights': [1.0, 1.0],
    'input_features': ['G(TS)_xtb', 'DrG_xtb'],
    'file_patterns': ['*_reactant.xyz', '*_ts.xyz', '*_product.xyz'],
    'file_dir_pattern': 'reaction_*',
    'id_field': 'ID',
    'dir_field': 'R_dir',
    'reaction_field': 'reaction',
    'cv_folds': 0, # Set > 0 for cross-validation
    'use_scaler': True,  # Controls whether to scale target values and pass scalers to trainer
    
    # Model parameters (DimeNet++ specific)
    'model_type': 'dimenet++',
    'node_dim': 128,
    'dropout': 0.1,
    'prediction_hidden_layers': 3,
    'prediction_hidden_dim': 512,
    'use_layer_norm': False,
    
    'hidden_channels': 128,
    'num_blocks': 5,
    'int_emb_size': 64,
    'basis_emb_size': 8,
    'out_emb_channels': 256,
    'num_spherical': 7,
    'num_radial': 6,
    'cutoff': 5.0,
    'envelope_exponent': 5,
    'num_before_skip': 1,
    'num_after_skip': 2,
    'num_output_layers': 3,
    'max_num_neighbors': 32,
    
    # Training parameters
    'batch_size': 16,
    'eval_batch_size': None, # Uses batch_size if None
    'lr': 0.0005,
    'finetune_lr': None,
    'epochs': 20,  # Set to 10 for demonstration
    'min_epochs': 0,
    'early_stopping': 40,
    'optimizer': 'adamw',
    'scheduler': 'warmup_cosine',
    'warmup_epochs': 10,
    'min_lr': 1e-7,
    'weight_decay': 0.0001,
    'random_seed': 42234,
    
    'out_dir': './results/reaction_model', # Adjust path if needed
    'save_best_model': True,
    'save_last_model': False,
    'checkpoint_path': None, # Path to a .ckpt file to resume/continue
    'mode': 'continue', # 'train' or 'continue'
    'freeze_base_model': False,
    
    'cuda': True, # Set to False to force CPU
    'gpu_id': 0,
    'num_workers': 4 # Number of workers for data loading
}

## 3. Set Up GPU and Output Directory

In [17]:
# Setup GPU or CPU
if params['cuda'] and torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = str(params['gpu_id'])
    device = torch.device(f"cuda:{params['gpu_id']}")
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    device = torch.device("cpu")
    print("Using CPU")
    params['cuda'] = False

# Create output directory
os.makedirs(params['out_dir'], exist_ok=True)
print(f"Output directory created/exists: {params['out_dir']}")

Using GPU: NVIDIA GeForce RTX 4090 D
Output directory created/exists: ./results/reaction_model


## 4. Create Configuration Object

Use the unified configuration interface to create the configuration from parameters dictionary.

In [18]:
# Create configuration directly from parameters dictionary
config = Config.from_params(params)
print("Configuration created successfully")

Configuration created successfully


## 5. Load and Prepare Dataset

Load the dataset with the unified configuration object.

In [19]:
# Load dataset using the unified configuration
print("Loading dataset from unified configuration")

# Pass the entire config object to the dataset
dataset = ReactionDataset(config=config)

print("Dataset loaded successfully")
data_stats = dataset.get_data_stats()
print(f"Dataset stats: Train: {data_stats['train_size']}, Validation: {data_stats['val_size']}, Test: {data_stats['test_size']}")
if config.reaction.cv_folds > 0:
    print(f"Cross-validation enabled with {dataset.get_num_folds()} folds.")

Loading dataset from unified configuration
Inference mode changed from True to False
Using target fields: ['G(TS)', 'DrG']
Using input features: ['G(TS)_xtb', 'DrG_xtb']
Using file patterns: ['*_reactant.xyz', '*_ts.xyz', '*_product.xyz']



Processing reactions:  76%|███████▌  | 1201/1582 [00:00<00:00, 1739.48it/s]


Processing reactions: 100%|██████████| 1582/1582 [00:00<00:00, 1696.51it/s]


Processed 1580 reactions, skipped 2 reactions
Saved metadata to dataset/DATASET_DA_F/processed/metadata.json
Processed 1580 reactions, saved to dataset/DATASET_DA_F/processed/data_038b0f2fed6b.pt
Dataset split: train 1269, validation 162, test 149 samples
Dataset loaded successfully
Dataset stats: Train: 1269, Validation: 162, Test: 149


## 6. Initialize and Train Model

Create a trainer and train the model with the simplified interface.

In [20]:
# Create trainer - use scaler based on the config parameter
scalers = dataset.get_scalers() if config.reaction.use_scaler else None

print(f"Using scaler: {config.reaction.use_scaler}")

trainer = ReactionTrainer(
    config=config,
    scalers=scalers  # Pass scalers only if use_scaler is True
)

print("Trainer initialized successfully")
print(f"Starting training with {config.training.max_epochs} epochs")

Seed set to 42234


Using scaler: True
Trainer initialized successfully
Starting training with 20 epochs


In [21]:
# Train the model
train_metrics = trainer.fit(
    train_dataset=dataset.train_data,
    val_dataset=dataset.val_data,
    test_dataset=dataset.test_data,
    checkpoint_path=config.training.resume_from_checkpoint,
    mode=config.training.mode
)

print("Training completed successfully")
print("Metrics:", train_metrics)
if 'best_model_path' in train_metrics and train_metrics['best_model_path']:
    print(f"Best model saved to: {train_metrics['best_model_path']}")
elif config.training.save_last_model and 'last_model_path' in train_metrics and train_metrics['last_model_path']:
    print(f"Last model saved to: {train_metrics['last_model_path']}")

Using explicitly provided num_targets: 2
Using provided target weights: [1.0, 1.0]
Using target field names: ['G(TS)', 'DrG']
Final model configuration: num_targets=2, target_field_names=['G(TS)', 'DrG'], target_weights=[1.0, 1.0]
Initializing model with output_dim=2


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/root/miniconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /root/DeepReaction/results/reaction_model/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | model          | MoleculePredictionModel   | 3.5 M  | train
1 | net            | DimeNetPlusPlus           | 2.3 M  | train
2 | readout_module | MeanReadout               | 0      | train
3 | regr_or_cls_nn | MultiTargetPredictionHead | 1.2 M  | train
---------------------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.866    Total estimated model params size (MB)
193       Modules in train mode
0         Modules in

DEBUG - MultiTargetPredictionHead initialized with 2 targets
DEBUG - MultiTargetPredictionHead input_dim: 130
DEBUG - Creating head for target 0
DEBUG - Creating head for target 1
Epoch 0: 100%|██████████| 80/80 [00:26<00:00,  2.97it/s, v_num=1, train_total_loss_step=0.597]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 80/80 [00:29<00:00,  2.70it/s, v_num=1, train_total_loss_step=0.597, val_total_loss_step=0.203, val_total_loss_epoch=0.432, train_total_loss_epoch=1.440]

Metric val_total_loss improved. New best score: 0.432
Epoch 0, global step 80: 'val_total_loss' reached 0.43250 (best 0.43250), saving model to '/root/DeepReaction/results/reaction_model/checkpoints/best-epoch=0000-val_total_loss=0.4325.ckpt' as top 1


Epoch 1: 100%|██████████| 80/80 [00:27<00:00,  2.93it/s, v_num=1, train_total_loss_step=0.101, val_total_loss_step=0.203, val_total_loss_epoch=0.432, train_total_loss_epoch=1.440]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 80/80 [00:29<00:00,  2.67it/s, v_num=1, train_total_loss_step=0.101, val_total_loss_step=0.140, val_total_loss_epoch=0.206, train_total_loss_epoch=0.421]

...

Epoch 19: 100%|██████████| 80/80 [00:27<00:00,  2.95it/s, v_num=1, train_total_loss_step=0.0382, val_total_loss_step=0.0493, val_total_loss_epoch=0.0528, train_total_loss_epoch=0.0457]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|██████████| 80/80 [00:29<00:00,  2.69it/s, v_num=1, train_total_loss_step=0.0382, val_total_loss_step=0.0355, val_total_loss_epoch=0.0523, train_total_loss_epoch=0.0437]

Epoch 19, global step 1600: 'val_total_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 80/80 [00:29<00:00,  2.69it/s, v_num=1, train_total_loss_step=0.0382, val_total_loss_step=0.0355, val_total_loss_epoch=0.0523, train_total_loss_epoch=0.0437]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 10/10 [00:02<00:00,  4.78it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test Avg MAE          1.5185050964355469
     Test Avg MAX_AE        10.971177101135254
   Test Avg MEDIAN_AE       1.1011011600494385
      Test Avg MPAE          33.17732238769531
       Test Avg R2          0.9511441588401794
      Test Avg RMSE          2.188504457473755
      Test MAE DrG          1.7905664443969727
     Test MAE G(TS)          1.246443748474121
     Test MAX_AE DrG        16.361448287963867
    Test MAX_AE G(TS)        5.580905914306641
   Test MEDIAN_AE DrG        1.32346773147583
  Test MEDIAN_AE G(TS)      0.8787345886230469
      Test MPAE DrG          62.69510269165039
     Test MPAE G(TS)        3.6595